In [80]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

In [81]:
dados=pd.read_csv("DadosDeCancer.csv")


In [82]:
dados2=pd.DataFrame(np.copy(dados),columns=dados.columns)

In [83]:
dados2.head()

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,no


# A única variável que é numérica está com o seu tipo alterado pois veio de um array numpy, irei consertar em seguida.

In [84]:
dados2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 9 columns):
class         286 non-null object
age           286 non-null object
menopause     286 non-null object
tumor-size    286 non-null object
inv-nodes     286 non-null object
node-caps     286 non-null object
deg-malig     286 non-null object
breast        286 non-null object
irradiat      286 non-null object
dtypes: object(9)
memory usage: 20.2+ KB


In [85]:
#Ajeitando o tipo da variável deg-malig
dados2["deg-malig"]=np.array([int(i) for i in dados2["deg-malig"]])

In [86]:
dados2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 9 columns):
class         286 non-null object
age           286 non-null object
menopause     286 non-null object
tumor-size    286 non-null object
inv-nodes     286 non-null object
node-caps     286 non-null object
deg-malig     286 non-null int32
breast        286 non-null object
irradiat      286 non-null object
dtypes: int32(1), object(8)
memory usage: 19.1+ KB


# Mudando as classes para 0 e 1.

In [87]:
dados2["Classe"]=np.array([1 if i=="no-recurrence-events" else 0 for i in dados2["class"]])

In [88]:
dados2["Classe"]

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
256    0
257    0
258    0
259    0
260    0
261    0
262    0
263    0
264    0
265    0
266    0
267    0
268    0
269    0
270    0
271    0
272    0
273    0
274    0
275    0
276    0
277    0
278    0
279    0
280    0
281    0
282    0
283    0
284    0
285    0
Name: Classe, Length: 286, dtype: int32

# Usando a transformação OneHotEncoder para mudar as variáveis categóricas em numéricas, se baseia que em cada tipo único de cada variável fazer uma coluna de zeros e uns, 1 se o tipo único acontecer ou 0 caso contrário.

In [89]:
transformacao=OneHotEncoder(sparse=False)

# Organizando os dados necessários para o Perceptron, em que a primeira coluna é o peso do bias.

In [90]:
dados2=np.column_stack((np.full(dados2.shape[0],-1),transformacao
.fit_transform(dados2.loc[:,["age","menopause","tumor-size","inv-nodes"
,"node-caps","breast","irradiat"]]),dados2["deg-malig"],dados2["Classe"]))

In [91]:
x=dados2[:,:-1]
y=dados2[:,-1]

In [92]:
xtreino,xteste,ytreino,yteste=train_test_split(x,y,test_size=0.3,stratify=y)

# Exemplos de modelos

In [93]:
# Função de ativação identidade, ou seja, a saída para as demais redes é uma função linear, 
#o método de atualização dos pesos por gradiente descente estocástico, uma camada oculta com 3 neurônios,
#a tolerâncias de convergência dos pesos é 0.00001 e a taxa de aprendizado constante para todas as optimizações
# dos pesos e no máximo 20000 épocas até a convergência.
modelo=MLPClassifier(activation="identity",solver="sgd",
hidden_layer_sizes=(3),tol=0.00001,learning_rate="constant",max_iter=20000) 

In [94]:
modelo.fit(xtreino,ytreino)

MLPClassifier(activation='identity', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=3, learning_rate='constant',
       learning_rate_init=0.001, max_iter=20000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='sgd', tol=1e-05,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [95]:
metrics.accuracy_score(yteste,modelo.predict(xteste))

0.6976744186046512

In [96]:
print(metrics.classification_report(yteste, modelo.predict(xteste)))

              precision    recall  f1-score   support

         0.0       0.50      0.27      0.35        26
         1.0       0.74      0.88      0.80        60

   micro avg       0.70      0.70      0.70        86
   macro avg       0.62      0.58      0.58        86
weighted avg       0.66      0.70      0.67        86



In [97]:
# Função de ativação identidade, ou seja, a saída para as demais redes é uma função linear, 
#o método de atualização dos pesos por gradiente descente estocástico, dois camada ocultas com 3 neurônios a primeira,
#e a segunda camada oculta com 2 neurônios
#a tolerâncias de convergência dos pesos é 0.00001 e a taxa de aprendizado constante para todas as optimizações
# dos pesos e o máximo de épocas até a convergência é 20000.
modelo=MLPClassifier(activation="identity",solver="sgd",
hidden_layer_sizes=(3,2),tol=0.00001,learning_rate="constant",max_iter=20000)

In [98]:
modelo.fit(xtreino,ytreino)

MLPClassifier(activation='identity', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(3, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=20000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='sgd', tol=1e-05,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [99]:
metrics.accuracy_score(yteste,modelo.predict(xteste))

0.7093023255813954

In [100]:
print(metrics.classification_report(yteste, modelo.predict(xteste)))

              precision    recall  f1-score   support

         0.0       0.55      0.23      0.32        26
         1.0       0.73      0.92      0.81        60

   micro avg       0.71      0.71      0.71        86
   macro avg       0.64      0.57      0.57        86
weighted avg       0.68      0.71      0.67        86

